# 1. Install and Import Baseline Dependencies

In [1]:
!pip install transformers

In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

/Users/lomagnoc/Documents/Backup from Old Mac/Projects/deep-reinforcement-learning-trader/venv/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lomagnoc/Documents/Backup from Old Mac/Projects/deep-reinforcement-learning-trader/venv/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lomagnoc/Documents/Backup from Old Mac/Projects/deep-reinforcement-learning-trader/venv/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated;

KeyboardInterrupt: 

# 2. Setup Summarization Model

In [ ]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# 3. Summarize a Single Article

In [ ]:
url = "https://au.finance.yahoo.com/news/china-restricting-tesla-use-uncovers-a-significant-challenge-for-elon-musk-expert-161921664.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs[0].text

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [ ]:
ARTICLE

In [ ]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
summary

# 4. Building a News and Sentiment Pipeline

In [ ]:
monitored_tickers = ['GME', 'TSLA', 'BTC']

## 4.1. Search for Stock News using Google and Yahoo Finance

In [ ]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [ ]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

In [ ]:
raw_urls['GME']

## 4.2. Strip out unwanted URLs

In [ ]:
import re

In [ ]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

## 4.3. Search and Scrape Cleaned URLs

In [ ]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [ ]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

In [ ]:
articles['TSLA'][2]

## 4.4. Summarise all Articles

In [ ]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [ ]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

In [ ]:
summaries['BTC']

# 5. Adding Sentiment Analysis

In [ ]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

In [ ]:
sentiment(summaries['BTC'])

In [ ]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

In [ ]:
print(summaries['GME'][3], scores['GME'][3]['label'], scores['GME'][3]['score'])

In [ ]:
scores['BTC'][0]['score']

# 6. Exporting Results to CSV

In [ ]:
summaries

In [ ]:
scores

In [ ]:
cleaned_urls

In [ ]:
range(len(summaries['GME']))

In [ ]:
summaries['GME'][3]

In [ ]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [ ]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

In [ ]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [ ]:
final_output

In [ ]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)